Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic Regression with L2

In [5]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights)) #L2 Regularization at set Beta value
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.412441
Minibatch accuracy: 12.5%
Validation accuracy: 11.8%
Minibatch loss at step 500: 2.622232
Minibatch accuracy: 73.4%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.822093
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 1.058701
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 0.895611
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.855382
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 0.713988
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Test accuracy: 88.2%


Neural Network with Hidden Layers and L2

In [9]:
batch_size = 128
beta = 0.001
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.Adding Hidden Layer on Level 2.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layers]))
  biases_1 = tf.Variable(tf.zeros([hidden_layers]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layers, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  def forward_prop(train_data):
        '''Calculate logit with Rectified Linear Units'''
        hidden_layer_1 = tf.nn.relu(tf.matmul(train_data, weights_1) + biases_1)
        output_layer = tf.matmul(hidden_layer_1, weights_2) + biases_2
        return output_layer

  # Training computation.
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights_1)  #L2 Regularization on Weights_1
    + beta*tf.nn.l2_loss(weights_2)) #L2 Regularization on Weights_2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 642.593994
Minibatch accuracy: 8.6%
Validation accuracy: 31.9%
Minibatch loss at step 500: 196.500137
Minibatch accuracy: 84.4%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 114.041084
Minibatch accuracy: 78.1%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 68.757599
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 41.245598
Minibatch accuracy: 89.8%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 25.335655
Minibatch accuracy: 85.2%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 15.515965
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
batch_size = 128
beta = 0.001
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.Adding Hidden Layer on Level 2.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layers]))
  biases_1 = tf.Variable(tf.zeros([hidden_layers]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layers, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  def forward_prop(train_data):
        '''Calculate logit with Rectified Linear Units'''
        hidden_layer_1 = tf.nn.relu(tf.matmul(train_data, weights_1) + biases_1)
        output_layer = tf.matmul(hidden_layer_1, weights_2) + biases_2
        return output_layer

  # Training computation.
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights_1)  #L2 Regularization on Weights_1
    + beta*tf.nn.l2_loss(weights_2)) #L2 Regularization on Weights_2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [12]:
num_steps = 3001
num_batches = 2

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 612.451599
Minibatch accuracy: 10.9%
Validation accuracy: 29.9%
Minibatch loss at step 500: 190.770493
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 1000: 115.693596
Minibatch accuracy: 100.0%
Validation accuracy: 64.8%
Minibatch loss at step 1500: 70.163002
Minibatch accuracy: 100.0%
Validation accuracy: 64.7%
Minibatch loss at step 2000: 42.550705
Minibatch accuracy: 100.0%
Validation accuracy: 64.6%
Minibatch loss at step 2500: 25.805080
Minibatch accuracy: 100.0%
Validation accuracy: 64.6%
Minibatch loss at step 3000: 15.649665
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Test accuracy: 70.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Without overfit

In [15]:
batch_size = 128
beta = 0.001
keep_prob = 0.7
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.Adding Hidden Layer on Level 2.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layers]))
  biases_1 = tf.Variable(tf.zeros([hidden_layers]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layers, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  def forward_prop(train_data):
        '''Calculate logit with Rectified Linear Units'''
        hidden_layer_1 = tf.nn.relu(tf.matmul(train_data, weights_1) + biases_1)
        output_layer = tf.matmul(hidden_layer_1, weights_2) + biases_2
        return output_layer

  # Training computation.
  logits = tf.nn.dropout(forward_prop(tf_train_dataset),keep_prob) #Dropout added
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights_1)  #L2 Regularization on Weights_1
    + beta*tf.nn.l2_loss(weights_2)) #L2 Regularization on Weights_2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 755.637939
Minibatch accuracy: 9.4%
Validation accuracy: 25.2%
Minibatch loss at step 500: 210.168900
Minibatch accuracy: 61.7%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 114.414879
Minibatch accuracy: 66.4%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 68.400772
Minibatch accuracy: 68.0%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 41.505020
Minibatch accuracy: 64.8%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.442825
Minibatch accuracy: 67.2%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 15.624242
Minibatch accuracy: 74.2%
Validation accuracy: 86.2%
Test accuracy: 91.9%


With Overfit using reduced batches

In [13]:
batch_size = 128
beta = 0.001
keep_prob = 0.7
hidden_layers = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.Adding Hidden Layer on Level 2.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layers]))
  biases_1 = tf.Variable(tf.zeros([hidden_layers]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layers, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  def forward_prop(train_data):
        '''Calculate logit with Rectified Linear Units'''
        hidden_layer_1 = tf.nn.relu(tf.matmul(train_data, weights_1) + biases_1)
        output_layer = tf.matmul(hidden_layer_1, weights_2) + biases_2
        return output_layer

  # Training computation.
  logits = tf.nn.dropout(forward_prop(tf_train_dataset),keep_prob) #Dropout added
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights_1)  #L2 Regularization on Weights_1
    + beta*tf.nn.l2_loss(weights_2)) #L2 Regularization on Weights_2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [14]:
num_steps = 3001
num_batches = 2

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 779.132507
Minibatch accuracy: 4.7%
Validation accuracy: 26.9%
Minibatch loss at step 500: 191.436172
Minibatch accuracy: 79.7%
Validation accuracy: 63.9%
Minibatch loss at step 1000: 116.194763
Minibatch accuracy: 80.5%
Validation accuracy: 63.9%
Minibatch loss at step 1500: 70.603226
Minibatch accuracy: 78.9%
Validation accuracy: 64.0%
Minibatch loss at step 2000: 42.922565
Minibatch accuracy: 80.5%
Validation accuracy: 64.1%
Minibatch loss at step 2500: 26.111122
Minibatch accuracy: 86.7%
Validation accuracy: 64.2%
Minibatch loss at step 3000: 15.911554
Minibatch accuracy: 87.5%
Validation accuracy: 64.5%
Test accuracy: 70.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [74]:
batch_size = 128
beta = 0.0001
n_hidden_nodes1 = 1024
n_hidden_nodes2 = 300
n_hidden_nodes3 = 50

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.Adding Hidden Layer on Level 2.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases_1 = tf.Variable(tf.zeros([n_hidden_nodes1]))
  weights_2 = tf.Variable(
    tf.truncated_normal([n_hidden_nodes1, n_hidden_nodes2],
        stddev=np.sqrt(2.0 / (n_hidden_nodes1))))
  biases_2 = tf.Variable(tf.zeros([n_hidden_nodes2]))
  weights_3 = tf.Variable(
    tf.truncated_normal([n_hidden_nodes2, n_hidden_nodes3],
        stddev=np.sqrt(2.0 / (n_hidden_nodes2))))
  biases_3 = tf.Variable(tf.zeros([n_hidden_nodes3]))
  weights_4 = tf.Variable(
    tf.truncated_normal([n_hidden_nodes3, num_labels],
        stddev=np.sqrt(2.0 / (n_hidden_nodes3))))
  biases_4 = tf.Variable(tf.zeros([num_labels]))
  
  def forward_prop(train_data):
        '''Calculate logit with Rectified Linear Units'''
        hidden_layer_1 = tf.nn.relu(tf.matmul(train_data, weights_1) + biases_1)
        hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1, weights_2) + biases_2)
        hidden_layer_3 = tf.nn.relu(tf.matmul(hidden_layer_2, weights_3) + biases_3)
        output_layer = tf.matmul(hidden_layer_3, weights_4) + biases_4
        return output_layer

  # Training computation.
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta*tf.nn.l2_loss(weights_1)  #L2 Regularization on Weights_1
    + beta*tf.nn.l2_loss(weights_2) #L2 Regularization on Weights_2
    + beta*tf.nn.l2_loss(weights_3) #L2 Regularization on Weights_3
    + beta*tf.nn.l2_loss(weights_4)) #L2 Regularization on Weights_4
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [75]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.542243
Minibatch accuracy: 7.0%
Validation accuracy: 34.6%
Minibatch loss at step 500: 0.476870
Minibatch accuracy: 90.6%
Validation accuracy: 85.9%
Minibatch loss at step 1000: 0.532280
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 1500: 0.422931
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 2000: 0.392170
Minibatch accuracy: 91.4%
Validation accuracy: 88.8%
Minibatch loss at step 2500: 0.397518
Minibatch accuracy: 93.0%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.454684
Minibatch accuracy: 87.5%
Validation accuracy: 89.6%
Test accuracy: 94.6%
